## Packages

In [1]:
import pandas as pd
import numpy as np
import random
import os
from scipy.stats import norm
from scipy.interpolate import interp1d
from scipy.optimize import minimize
from scipy import optimize
import sympy
from sympy import integrate
from sympy.solvers import solve
import warnings
import time
from datetime import datetime
warnings.filterwarnings('ignore')

## HyperParameters

In [2]:
q = 0.01799
Underlying = 'CNTBI10new'
N = 5   #number of lognormals
sampleDeltaChoice = [0,2,6,8]  #选择几个delta作为模型训练
totalTrials = 100

## Functions

In [3]:
def StrikeFromFwdDelta(CallPutFlag, Delta, F, Sigma, T):
    if CallPutFlag == 1:
        w  = 1
    else:
        w = -1
    
    K = F * np.exp(-w * Sigma * np.sqrt(T) * norm.ppf(w * Delta) + 0.5 * Sigma * Sigma * T)
    
    return K

def BS(S0, K, r, T, Sigma, q, call = True):
    d1 = (np.log(S0/K) + (r -q + 0.5 * Sigma**2) * T) / (Sigma * np.sqrt(T))
    d2 = d1 - Sigma * np.sqrt(T)
    if call:
        priceBS = np.exp(-q * T) * S0 * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)
    else:
        priceBS = -np.exp(-q * T) * S0 * norm.cdf(-d1) + np.exp(-r * T) * K * norm.cdf(-d2)
    
    return priceBS

def erfMC(a,b):
    a = 0
    b = b
    t = np.random.rand(1000000)
    z = a + (b - a) * t
    ht = (b - a) * np.exp(- z ** 2)
    erf = 2 / np.sqrt(np.pi) * np.mean(ht)
    return erf
    
def NormalCDF(x, mu = 0, sigma = 1):
    CDF = 0.5 * (1 + erfMC(a = 0, b = (x - mu) / sigma / np.sqrt(2)))
    return CDF
    

def SolveIV(S0, K, r, T, q,modelPrice, call = True, ):
    def f(Sigma):
        d1 = (np.log(S0/K) + (r -q + 0.5 * Sigma**2) * T) / (Sigma * np.sqrt(T))
        d2 = d1 - Sigma * np.sqrt(T)
        #print(d1, d2)
        if call:
            priceBS = np.exp(-q * T) * S0 * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)
        else:
            priceBS = -np.exp(-q * T) * S0 * norm.cdf(-d1) + np.exp(-r * T) * K * norm.cdf(-d2)
        #print(priceBS)
        #IV = solve(priceBS, Sigma)
        return (priceBS - modelPrice)
    IV = optimize.bisect(f,-10,10)
    return IV

def GetImpliedDividendYield(S0, r, T, Fwd):
    q = np.log(S0 * np.exp(r * T) / Fwd) / T
    return q

def ModelPrice(params, date, K, call = True):
    pT = params[:N]
    eplisonT = params[N:(2*N)]
    SigmaT = params[(2*N):(3*N)]
    QT = params[-1]
    #print(pT,eplisonT,SigmaT,QT)
    
    fwd = fwdMap[date]
    T = ((date - valuationDate).days)/365
    r = interpolate_riskFree(T)
    #q = GetImpliedDividendYield(spotPrice, r, T, fwd)
    modelPrice = 0
    if call:
        for i in range(N):
            modelPrice += pT[i] * BS(eplisonT[i] * spotPrice, K, r, T, SigmaT[i]/np.sqrt(T), q, call = True )
    else:
        for i in range(N):
            modelPrice += pT[i] * BS(eplisonT[i] * spotPrice, K, r, T, SigmaT[i]/np.sqrt(T), q, call = False) + QT * K * np.exp(-r * T)

    return modelPrice

def Error(x, y, method = 0):
    
    if method == 0:
        error = (x - y) ** 2
    elif method == 1:
        error = ((x - y) / y) ** 2
    else:
        error = abs((x - y) / y)
    return error
        

def AllStrikeError(params, date, mktOptionPrice_date):
    error = 0
    for i, column in enumerate(mktOptionPrice_date.columns):
        delta = column[0]
        mktStrike = column[1]
        mktOptionPrice = mktOptionPrice_date.iloc[:,i].values
        modelPrice = ModelPrice(params, date, mktStrike, call = (delta>0))
        error += Error(modelPrice, mktOptionPrice, method = 1)
        #print(delta, mktStrike)
    error = np.sqrt(error / len(sampleDeltaChoice))
    return error[0]

## Optimization

In [4]:
def Objective(params, mktOptionPrice_date, date, returnType = 0):
    def Objective2(params):
        mktOptionPrice_date_sample = mktOptionPrice_date.iloc[:,sampleDeltaChoice]
        error = 0
        for i, column in enumerate(mktOptionPrice_date_sample.columns):
            delta = column[0]
            mktStrike = column[1]
            mktOptionPrice = mktOptionPrice_date_sample.iloc[:,i].values
            modelPrice = ModelPrice(params, date, mktStrike, call = (delta>0))
            #print(modelPrice)
            error += Error(modelPrice, mktOptionPrice, method = 1)
            #print(delta, mktStrike)
        error = np.sqrt(error / len(sampleDeltaChoice))
        return error[0]
    
    if returnType == 0:
        return Objective2
    else :
        return Objective2(params)

def Constraints(params, returnType = 0):

    def Constraints1(params):
        pT = params[:N]
        eplisonT = params[N:(2*N)]

        return np.dot(pT, eplisonT) - 1

    def Constraints2(params):
        pT = params[:N]
        QT = params[-1]
        return sum(pT) + QT - 1

    cons = [{'type':"eq",'fun':Constraints1},
           {'type':"eq",'fun':Constraints2}]
    
    if returnType == 0:
        return cons
    else :
        return Constraints1(params),Constraints2(params)

def Bounds(PARAMS, maturityChoice):
    bounds = []
    for i in range(N):
        bounds.append((0,1))
    for i in range(N):
        bounds.append((0,10))
        
    if maturityChoice == 0:
        for i in range(N):
            bounds.append((0,10))
        bounds.append((0,1))
    else:
        paramsPrevios = PARAMS[maturityChoice - 1]
        SigmaT = paramsPrevios[(2*N):(3*N)]
        QT = paramsPrevios[-1]
        
        for i in range(N):
            bounds.append((SigmaT[i],10))
        bounds.append((QT,1))   
        
    bounds = tuple(bounds)
    
    return bounds

## Read Data

In [5]:
root = os.getcwd()
input_path = os.path.join(root,'input')
output_path = os.path.join(root,'output')
filename = f'data_{Underlying}.xlsx'
file_path = os.path.join(input_path,filename)

In [6]:
data = pd.read_excel(file_path,sheet_name='data')

info = pd.read_excel(file_path,sheet_name='info',header = None)
valuationDate = pd.to_datetime(str(int(info.iloc[0,1])))
spotPrice = info.iloc[1,1]

deltaMap = pd.read_excel(file_path,sheet_name='deltaMap')
riskFree = pd.read_excel(file_path,sheet_name='riskFree')
deltaMap = deltaMap.set_index('name')['delta'].to_dict()
columns = list(data.columns.map(deltaMap))
columns[:2] = ['date','Fwd']
data.columns = columns
data['Fwd'].replace(' ',np.nan, inplace = True)
data.fillna(method = 'ffill',inplace = True)
data.set_index('date',inplace = True)

#data

## Get Market Data

In [7]:
fwdMap = data['Fwd'].to_dict()
dateList = list(data.index.unique())

mktImpliedVol = data.iloc[::2,1:]
#mktFwdPrice = data.iloc[1::2,1:]
mktOptionPrice = data.iloc[1::2,1:]
mktOptionPrice.iloc[:,:] = np.nan

In [8]:
#Risk Free interpolate
x = riskFree['Date'].values
y = riskFree['Zero Rate'].values/100
#y = np.log(1 + y)
interpolate_riskFree = interp1d(x, y)


In [9]:
mktImpliedVol_date_List = []
#mktFwdPrice_date_List = []
mktOptionPrice_date_List = []
mktStrikeDict={}
for date in dateList:
    mktStrikeList = []
    
    
    mktOptionPrice_date = mktOptionPrice.iloc[mktOptionPrice.index == date]
    #mktFwdPrice_date = mktFwdPrice.iloc[mktFwdPrice.index == date]
    mktImpliedVol_date = mktImpliedVol.iloc[mktImpliedVol.index == date]
    
    fwd = fwdMap[date]
    T = ((date - valuationDate).days)/365
    r = interpolate_riskFree(T)
    #q = GetImpliedDividendYield(spotPrice, r, T, fwd)
    
    for Delta in mktImpliedVol.columns:

        Sigma = mktImpliedVol.loc[date,Delta]
        #print(Sigma)
        mktStrike = StrikeFromFwdDelta(np.sign(Delta), Delta, fwd, Sigma, T)
        #print(mktStrike)
        mktStrikeList.append(mktStrike)
        
        priceBS = BS(spotPrice, mktStrike, r, T, Sigma, q, call = (Delta > 0))
        #print(priceBS)
        mktOptionPrice.loc[date, Delta] = priceBS
        mktOptionPrice_date.loc[date, Delta] = priceBS
        
    #print(mktStrikeList)
    columnArray = np.array([list(mktImpliedVol_date.columns),mktStrikeList])
    #print(columnArray)
    
    mktStrikeDict[date] = mktStrikeList
    
    mktImpliedVol_date.columns = pd.MultiIndex.from_arrays(columnArray, names = ['Delta','mktStrike'])
    #mktFwdPrice_date.columns = pd.MultiIndex.from_arrays(columnArray, names = ['Delta','mktStrike'])
    mktOptionPrice_date.columns = pd.MultiIndex.from_arrays(columnArray, names = ['Delta','mktStrike'])
    
    
    mktImpliedVol_date_List.append(mktImpliedVol_date)
    #mktFwdPrice_date_List.append(mktFwdPrice_date)
    mktOptionPrice_date_List.append(mktOptionPrice_date)

## Prepare Model Data

In [10]:
modelImpliedVol = mktImpliedVol.copy(deep = True)
modelImpliedVol.iloc[:,:] = np.nan
modelOptionPrice = mktOptionPrice.copy(deep = True)
modelOptionPrice.iloc[:,:] = np.nan

## Calibration

In [11]:
#Set Parameters (N个p(T),N个eplison(T),N个Sigma(T),Q(T))
PARAMS = np.zeros((len(dateList), 3 * N + 1))
PARAMS


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [12]:
objective_dict = {}
for maturityChoice in range(len(dateList)):
    date =dateList[maturityChoice]
    fwd = fwdMap[date]
    T = ((date - valuationDate).days)/365
    r = interpolate_riskFree(T)
    #q = GetImpliedDividendYield(spotPrice, r, T, fwd)
    
    mktOptionPrice_date = mktOptionPrice_date_List[maturityChoice]
    print(date)
    objective_dict[date] = {}
    
    for trial in range(totalTrials):
        print("trial",trial)   
        params_initial = np.random.random((1,N * 3 + 1))[0]
        #minimize()
        result = minimize(Objective(params_initial, mktOptionPrice_date, date, returnType = 0),
                          params_initial,method='SLSQP',
                          bounds=Bounds(PARAMS, maturityChoice),
                          constraints=Constraints(params_initial, returnType = 0))
        params = result.x
        objectiveResult = Objective(params, mktOptionPrice_date, date, returnType = 1)
        objective_dict[date][trial] = [objectiveResult, params]
        print(objectiveResult,AllStrikeError(params, date, mktOptionPrice_date))
        #print(params)
        print("====="*5)

    #print(objective_dict[date])
    objective_dict[date] = sorted(objective_dict[date].items(), key = lambda x: x[1][0])
    params = objective_dict[date][0][1][1]
    PARAMS[maturityChoice] = params

    for i, columns in enumerate(mktOptionPrice_date):
        delta = columns[0]
        mktStrike = columns[1] 
        modelPrice = ModelPrice(params, date, mktStrike, call = (delta > 0))
        modelOptionPrice.iloc[maturityChoice,i] = modelPrice
        modelImpliedVol.iloc[maturityChoice,i] = SolveIV(spotPrice, mktStrike, r, T, q, modelPrice, call = (delta > 0))
        
    print("#"*50)

2022-06-17 00:00:00
trial 0
0.7805275023351145 1.1855392171909245
trial 1
0.0005032415578436263 0.06126039205331203
trial 2
1.2787731334136782e-06 1.7799933712001637e-06
trial 3
1.9684325367473503e-05 2.703436805150265e-05
trial 4
2.9295128910002237e-05 4.609326288977541e-05
trial 5
0.7805595417034066 1.1855727172106678
trial 6
0.00022503335665363964 0.00044079759287075893
trial 7
5.702364571076355e-07 7.114290730369174e-07
trial 8
5.009659466831195e-05 7.015934695254345e-05
trial 9
0.00018964290582834756 0.0007115023156108705
trial 10
1.373085115160782e-05 1.931672126827509e-05
trial 11
0.0008815714220241886 0.0022460627057058033
trial 12
3.23182545170162e-05 4.383456031264416e-05
trial 13
0.00026493476921407116 0.10083504616454507
trial 14
1.1372150692801386e-05 1.8215668082712478e-05
trial 15
0.7805590694208964 1.185434637088089
trial 16
1.8840489976250748e-05 2.9815994088814055e-05
trial 17
3.067191121224947e-05 5.209893182040219e-05
trial 18
8.627260029367506e-05 0.000403234831412

2.8186597722568642e-05 4.327561301880711e-05
trial 5
1.212475374675734e-05 1.3557958793260609e-05
trial 6
1.0879224168538543e-06 1.3519025112316877e-06
trial 7
0.00011369874939304161 0.09034645821382138
trial 8
1.440164337949291e-06 1.6825340384220583e-06
trial 9
2.252802429069099e-06 2.6664428508722307e-06
trial 10
1.691416285801533e-07 1.9961286184596465e-07
trial 11
0.0002052400715142362 0.007435030087119462
trial 12
6.808513761177643e-08 1.0734552107107583e-07
trial 13
8.15736971048685e-06 9.635990821419594e-06
trial 14
2.1762887338613458e-05 2.482785543725884e-05
trial 15
0.00011850231800616132 0.0044899370166780524
trial 16
0.0007414607888594401 0.0008410032838146994
trial 17
2.762423704036212e-06 0.006174030191446254
trial 18
0.4573382837194153 0.8108544956916747
trial 19
1.6483989516053637e-05 2.4446022080031006e-05
trial 20
2.8256597441606534e-05 3.730862382096438e-05
trial 21
1.993462489891662e-06 0.00280891033620676
trial 22
1.82526634913121e-06 2.417084177406272e-06
trial 2

2.0232150211003337e-06 2.617035907644509e-06
trial 8
1.132340160012722e-06 1.469009970561738e-06
trial 9
3.2638667155427396e-06 4.714103717550969e-06
trial 10
1.0477086113957591e-06 1.3676083716000593e-06
trial 11
6.609459580083053e-07 8.63323558071425e-07
trial 12
2.11562765913503e-05 2.3050745423835692e-05
trial 13
1.0331837614511022e-05 1.1532559859297427e-05
trial 14
97.89762564356066 115.44702395060996
trial 15
8.774756373325445e-05 0.02309314477452707
trial 16
7.449463870981138e-08 8.839215381021742e-08
trial 17
1.578052216494436e-06 2.1806849953126166e-06
trial 18
9.251216436881262e-06 1.032888901863428e-05
trial 19
1.8983647531703944e-05 0.0014514390044113698
trial 20
9.425639255981254e-06 1.164056174835375e-05
trial 21
4.4979318294190335e-07 5.540454614552905e-07
trial 22
5.595366364551173e-07 7.277832133562674e-07
trial 23
4.5422923278313876e-05 6.808926561479069e-05
trial 24
7.641755374562125e-08 9.337781665131217e-08
trial 25
3.03591935882742e-05 0.0001111586293899394
trial

3.6956258597220584e-06 4.85078074798939e-06
trial 11
1.2832987499376092e-06 1.6435509169710338e-06
trial 12
4.410557130470698e-07 5.516251843134469e-07
trial 13
4.159709539187739e-07 4.874366883842611e-07
trial 14
1.1366485621565102e-05 1.5325556824336056e-05
trial 15
2.8982574106374133e-05 0.0010776288907016944
trial 16
1.7163910854281897e-05 2.804581447617583e-05
trial 17
2.8854148165149193e-06 3.3434589781352063e-06
trial 18
7.663698689782972e-06 1.1535503912052589e-05
trial 19
1.9461810331638076e-05 3.180165103102878e-05
trial 20
1.669229745688099e-07 2.19647760267664e-07
trial 21
3.2127508039488403e-06 4.0785521186589115e-06
trial 22
8.887641151581205e-07 1.173155458560992e-06
trial 23
6.37886011269736e-06 8.418453322314275e-06
trial 24
6.15335594237044e-08 7.380835992466668e-08
trial 25
4.13513147127633e-06 4.803559499094241e-06
trial 26
5.067882703162933e-05 0.00017200544971601234
trial 27
6.323995764048042e-08 7.776237358682532e-08
trial 28
1.0210605694799335e-05 1.354585197207

In [13]:
mktImpliedVol

,-0.1,-0.15,-0.25,-0.35,0.5,0.35,0.25,0.15,0.1
date,,,,,,,,,
2022-06-17,0.0925,0.0925,0.0925,0.0925,0.0925,0.0925,0.0925,0.0925,0.0925
2022-06-24,0.0925,0.0925,0.0925,0.0925,0.0925,0.0925,0.0925,0.0925,0.0925
2022-07-10,0.1126,0.1126,0.1126,0.1126,0.1126,0.1126,0.1126,0.1126,0.1126
2022-08-10,0.0976,0.0976,0.0976,0.0976,0.0976,0.0976,0.0976,0.0976,0.0976


## Result

In [14]:
modelOptionPriceDiff = (modelOptionPrice/mktOptionPrice - 1)
modelOptionPriceDiff.applymap(lambda x: format(x, '.2%'))

,-0.1,-0.15,-0.25,-0.35,0.5,0.35,0.25,0.15,0.1
date,,,,,,,,,
2022-06-17,-0.00%,-0.00%,-0.00%,-0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
2022-06-24,0.00%,-0.00%,-0.00%,-0.00%,0.00%,0.00%,0.00%,0.00%,-0.00%
2022-07-10,0.00%,-0.00%,-0.00%,-0.00%,0.00%,0.00%,0.00%,-0.00%,-0.00%
2022-08-10,0.00%,0.00%,-0.00%,-0.00%,0.00%,-0.00%,-0.00%,-0.00%,-0.00%


In [15]:
modelImpliedVolDiff = (modelImpliedVol/mktImpliedVol - 1 )
modelImpliedVolDiff.applymap(lambda x: format(x, '.2%'))

,-0.1,-0.15,-0.25,-0.35,0.5,0.35,0.25,0.15,0.1
date,,,,,,,,,
2022-06-17,-0.00%,-0.00%,-0.00%,-0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
2022-06-24,0.00%,-0.00%,-0.00%,-0.00%,0.00%,0.00%,0.00%,0.00%,-0.00%
2022-07-10,0.00%,-0.00%,-0.00%,-0.00%,0.00%,0.00%,0.00%,-0.00%,-0.00%
2022-08-10,0.00%,0.00%,-0.00%,-0.00%,0.00%,-0.00%,-0.00%,-0.00%,-0.00%


In [19]:
modelImpliedVolDiff

,-0.1,-0.15,-0.25,-0.35,0.5,0.35,0.25,0.15,0.1
date,,,,,,,,,
2022-06-17,-4.093247e-08,-9.474015e-08,-1.547176e-07,-1.982751e-07,2.157663e-07,1.437933e-07,1.081018e-07,7.774430e-08,6.417562e-08
2022-06-24,6.115442e-09,-2.588894e-08,-5.821287e-08,-7.868879e-08,5.267174e-08,2.614891e-08,1.262939e-08,4.372458e-10,-5.757151e-09
2022-07-10,9.661745e-09,-6.613849e-09,-2.276828e-08,-3.268307e-08,2.159583e-08,9.964640e-09,4.007693e-09,-1.363657e-09,-4.109911e-09
2022-08-10,1.533200e-08,3.031454e-09,-8.360526e-09,-1.455739e-08,2.705303e-09,-2.885853e-09,-5.728024e-09,-8.313933e-09,-9.641833e-09


In [16]:
for maturityChoice in range(len(dateList)):
    date = dateList[maturityChoice]
    mktOptionPrice_date = mktOptionPrice_date_List[maturityChoice]
    print(date)
    params = PARAMS[maturityChoice]
    print("训练误差: %f, 总误差: %f" % (Objective(params, mktOptionPrice_date, date, returnType = 1),AllStrikeError(params, date, mktOptionPrice_date)))
    constraintResult = Constraints(params, returnType = 1)
    print("约束条件返回值：", constraintResult)
    print("约束条件是否满足：", all([abs(x) < 0.001 for x in constraintResult]))
    print("="*20)

2022-06-17 00:00:00
训练误差: 0.000000, 总误差: 0.000000
约束条件返回值： (2.770543794383684e-09, 8.380895977211367e-11)
约束条件是否满足： True
2022-06-24 00:00:00
训练误差: 0.000000, 总误差: 0.000000
约束条件返回值： (1.410717098693226e-09, 1.5629275651463104e-11)
约束条件是否满足： True
2022-07-10 00:00:00
训练误差: 0.000000, 总误差: 0.000000
约束条件返回值： (1.1136975786030234e-09, 5.402345237826012e-13)
约束条件是否满足： True
2022-08-10 00:00:00
训练误差: 0.000000, 总误差: 0.000000
约束条件返回值： (6.462401724860456e-10, 7.962519532611623e-12)
约束条件是否满足： True


In [17]:
PARAMS

array([[4.69109408e-12, 1.67116276e-11, 5.75099816e-12, 1.32109010e-12,
        1.00000000e+00, 4.68229750e+00, 2.05691323e+00, 4.47645601e-02,
        1.77190292e-01, 1.00000000e+00, 5.65116144e-01, 1.66008539e-03,
        2.46155988e-03, 5.46800250e-03, 1.28098572e-02, 6.06413158e-11],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        2.11939065e-14, 2.14732849e-01, 3.81564551e-01, 1.00000000e+00,
        1.27717318e-01, 1.25963143e-01, 5.65116144e-01, 1.75955485e-02,
        1.81158742e-02, 9.38906352e-03, 2.78498229e-02, 6.26542868e-11],
       [1.15464165e-14, 1.00000000e+00, 7.55573193e-15, 1.62220302e-16,
        4.24219700e-15, 4.00313046e-04, 1.00000000e+00, 1.57529030e-04,
        4.03532657e-04, 4.35487172e-04, 5.65228035e-01, 3.22814159e-02,
        1.84228870e-02, 9.91815438e-03, 2.81520232e-02, 6.27399037e-11],
       [1.57024750e-17, 1.06225229e-16, 5.92028578e-15, 2.39414214e-17,
        1.00000000e+00, 2.11396333e-02, 9.92523799e-03, 1.428

In [18]:
now = str(datetime.now().date())
filename = f'result_{Underlying}_{now}'
filterMask = [f for f in os.listdir(output_path) if f.startswith(filename)]
if len(filterMask) > 0:
    num = len(filterMask) + 1
else:
    num = 1
filename = filename + f"_v{num}.xlsx"
file_path = os.path.join(output_path,filename)
writer = pd.ExcelWriter(file_path, engine = 'openpyxl')
modelOptionPriceDiff.to_excel(writer, "modelOptionPriceDiff")
modelImpliedVolDiff.to_excel(writer, "modelImpliedVolDiff")
pd.DataFrame(PARAMS).to_excel(writer, "PARAMS")
writer.save()
writer.close()